# Kaggle 기본 예제

* 타이타닉 탑승자 데이터를 기반으로 생존자 예측하는 모델 생성

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### **[데이터 로딩]**

In [ ]:
titanic_df = pd.read_csv('./titanic_train.csv')
titanic_df.head()

### **[데이터 확인 및 전처리]**

In [ ]:
# 기본 요약 정보
titanic_df.info()

In [ ]:
titanic_df.Cabin.value_counts()

In [ ]:
# 머신러닝 모델은 NULL을 허용하지 않으므로 결측치 처리 수행 - 대체값
titanic_df['Age'].fillna(titanic_df['Age'].mean(), inplace=True)
titanic_df['Cabin'].fillna('N', inplace=True)
titanic_df['Embarked'].fillna('N', inplace=True)

print('NULL 갯수 : \n', titanic_df.isnull().sum())

In [ ]:
# Cabin 컬럼의 분포 확인
titanic_df.Cabin.value_counts()

In [ ]:
# Cabin Category를 크게 정리
titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]

titanic_df.Cabin.value_counts()

### **[모델의 예측 수행 전 데이터 탐색]**

* 어떤 유형의 승객이 생존확률이 더 높았을지 확인 (가설)


    1. 여성 또는 노약자 먼저 구한다.

    2. 부자 또는 유명인

    3. 삼등실 승객들


In [ ]:
# 성별에 따른 생존 결과 확인 (0 : 사망, 1 : 생존)
titanic_df.groupby(['Sex', 'Survived'])['Survived'].count()

In [ ]:
# 더 명확하게 확인하고 싶으면 시각화를 통해서 체크
sns.barplot(x='Sex', y='Survived', data=titanic_df)
plt.show()                               # 1 가설 중 여성이 상대적으로 더 많이 생존

In [ ]:
# 부자 또는 유명인의 생존 결과 확인
sns.barplot(x='Pclass', y='Survived', data=titanic_df)
plt.show()                   # 2, 3 가설 확인

In [ ]:
# 부자 또는 유명인의, 그리고 성별에 따른 생존 결과 확인
sns.barplot(x='Pclass', y='Survived', data=titanic_df, hue='Sex')
plt.show()

In [ ]:
# 1) 1 가설 중 노약자에 대한 확인이 필요
#   Age 컬럼은 연속된 숫자값 형태 (continous) -> 범위별로 구별되도록 처리
def get_category(age):
  category = ''
  if age <= -1 : category = 'Unknown'
  elif age <= 5 : category = 'Baby'
  elif age <= 12 : category = 'Child'
  elif age <= 18 : category = 'Teenager'
  elif age <= 25 : category = 'Student'
  elif age <= 35 : category = 'Young Adult'
  elif age <= 60 : category = 'Adult'
  else : category = 'Elderly'
  return category

In [ ]:
category_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Elderly']

titanic_df['Age_Category'] = titanic_df['Age'].apply(lambda x : get_category(x))

plt.figure(figsize=(12, 5))

sns.barplot(x='Age_Category', y='Survived', hue='Sex', data=titanic_df, order=category_names)
plt.show()

In [ ]:
titanic_df.head()

In [ ]:
titanic_df.drop('Age_Category', axis=1, inplace=True)

titanic_df.info()

### **[레이블 인코딩]**

In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode_feature(data):
  features = ['Sex', 'Embarked']
  for feature in features :
    encoder = LabelEncoder()
    data[feature] = encoder.fit_transform(data[feature])

  return data

In [ ]:
titanic_df = encode_feature(titanic_df)
titanic_df.head()

### **[머신러닝 전처리 작업을 한 번에 수행할 수 있도록 처리]**

In [ ]:
# 결측치 처리 함수
def fillna_nan(df):
  df['Age'].fillna(df['Age'].mean(), inplace=True)
  df['Cabin'].fillna('N', inplace=True)
  df['Embarked'].fillna('N', inplace=True)
  return df

# 레이블 인코딩
def label_encoding_features(df):
  features = ['Sex', 'Embarked']
  for feature in features :
    encoder = LabelEncoder()
    df[feature] = encoder.fit_transform(df[feature])
  return df

# 모델에게 불필요한 특성(feature) 제거
def drop_feature(df):
  df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
  return df

# 최종적으로 전처리를 수행할 함수
def transform_features(df):
  df = fillna_nan(df)
  df = drop_feature(df)
  df = label_encoding_features(df)
  return df

# [유의] 최종 전처리 함수 내부에서 수행될 개별 함수의 순서는 신경쓰기

* 최종 전처리 함수를 이용해서 데이터를 다시 가공

In [ ]:
# 데이터 로딩
titanic_df = pd.read_csv('./titanic_train.csv')

# target (label) 데이터
y_titanic = titanic_df['Survived']

# feature 데이터
X_titanic = titanic_df.drop('Survived', axis=1)

# feature 전처리
X_titanic_df = transform_features(X_titanic)

X_titanic_df.head()

In [ ]:
X_titanic_df.info()

### **[모델 생성 , 학습, 평가]**

In [ ]:
# 학습 세트, 테스트 세트 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic, test_size=0.2, random_state=11)

In [ ]:
# 예측 시 자주 사용하는 알고리즘 3가지로 각각 예측하고 평가
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 각각 모델 객체 생성
dt_cf = DecisionTreeClassifier(random_state=11)
rf_cf = RandomForestClassifier(random_state=11)
lr_crf = LogisticRegression(solver='liblinear', random_state=11)

In [ ]:
# 각각 모델 학습 및 예측한 결과를 (정확도) 평가
dt_cf.fit(X_train, y_train)
dt_cf_pred = dt_cf.predict(X_test)
print(f'결정트리 모델의 정확도 : {accuracy_score(y_test, dt_cf_pred)}')

rf_cf.fit(X_train, y_train)
rf_cf_pred = rf_cf.predict(X_test)
print(f'랜덤포레스트 모델의 정확도 : {accuracy_score(y_test, rf_cf_pred)}')

lr_crf.fit(X_train, y_train)
lr_crf_pred = lr_crf.predict(X_test)
print(f'로지스틱 회귀 모델의 정확도 : {accuracy_score(y_test, lr_crf_pred)}')
